# Protein quantification

In [ ]:
# Import packages 
import sys
import numpy as np
import pandas as pd
import seaborn as sns
from natsort import natsorted
from scipy.io import loadmat, savemat
from skimage.filters import threshold_otsu
from skimage.color import label2rgb
from tqdm.notebook import tqdm

import anndata as ad
import scanpy as sc
# Customized packages 
from starmap.sequencing import *

## Path

In [ ]:
# IO path 
base_path = 'Z:/Data/Analyzed/2022-09-05-Hu-Tissue/'

in_path = os.path.join(base_path, 'input')
if not os.path.exists(in_path):
    os.mkdir(in_path)
    
out_path = os.path.join(base_path, 'output')
if not os.path.exists(out_path):
    os.mkdir(out_path)
    
seg_path = os.path.join(in_path, "rep2/protein/")
if not os.path.exists(seg_path):
    os.mkdir(seg_path)
    
grid_path = os.path.join(seg_path, 'grid')
if not os.path.exists(grid_path):
    os.mkdir(grid_path)

In [ ]:
current_sample = 'STAR'

In [ ]:
# load h5ad object with cell locations 
cdata = sc.read_h5ad(os.path.join(out_path, '2022-11-20-Brain-combined-3mad-ct-final.h5ad'))
cdata

## Segmentation

In [ ]:
# get centroids
df = cdata.obs.loc[cdata.obs['protocol-replicate'] == f'{current_sample}map-rep2' , :].copy()
df['column_scaled'] = df['column'] * 0.25
df['row_scaled'] = df['row'] * 0.25
centroids = df.loc[:, ['column_scaled', 'row_scaled']].to_numpy()
centroids

In [ ]:
# load overlay image 
overlay = load_nissl_image(os.path.join(seg_path, 'segmentation'), fname=f"{current_sample}_overlay.tif")
# overlay = load_nissl_image(os.path.join(seg_path,), fname=f"test.tif")

In [ ]:
figsize=(overlay.shape[1] / 1000 * 5, overlay.shape[0] / 1000 * 5)

In [ ]:
# Plot dots on segmentation mask
plt.figure(figsize=figsize)
plt.imshow(overlay, cmap='gray')
plt.plot(centroids[:, 0], centroids[:, 1], '.', color='red', markersize=3)
plt.axis('off')
# plt.show()
points_seg_path = os.path.join(seg_path, f"{current_sample}_centroid.png")
print(f"Saving points_seg.png")
plt.savefig(points_seg_path)
# plt.clf()
# plt.close()

In [ ]:
%%time
# Segmentation

print("Gaussian & Thresholding")
blurred_overlay_seg = gaussian(overlay.astype(np.float), 2)
threhold = 32

# manual threshold 
blurred_overlay_seg = blurred_overlay_seg > threhold

# dialation  
blurred_overlay_seg = binary_dilation(blurred_overlay_seg, selem=disk(3))

print("Assigning markers")
centroids = centroids.astype(int)
markers = np.zeros(blurred_overlay_seg.shape, dtype=np.uint8)
for i in range(centroids.shape[0]):
    y, x = centroids[i, :]
    if x < blurred_overlay_seg.shape[0] and y < blurred_overlay_seg.shape[1]:
        markers[x-1, y-1] = 1
markers = ndi.label(markers)[0]

print("Watershed")
labels = watershed(blurred_overlay_seg, markers, mask=blurred_overlay_seg)
labels_line = watershed(blurred_overlay_seg, markers, mask=blurred_overlay_seg, watershed_line=True)

print(f"Labeled {len(np.unique(labels)) - 1} cells")
plt.figure(figsize=(10,20))
plt.imshow(label2rgb(labels_line, bg_label=0))

print(f"Saving files to {seg_path}")
tifffile.imsave(os.path.join(seg_path, f"{current_sample}_labeled_cells_line.tif"), labels_line.astype(np.uint16))
tifffile.imsave(os.path.join(seg_path, f"{current_sample}_labeled_cells.tif"), labels.astype(np.uint16))

In [ ]:
centroids.shape

## Quantification

In [ ]:
current_sample = 'STAR'
current_df = cdata.obs.loc[cdata.obs['protocol-replicate'] == f'{current_sample}map-rep2' , :].copy()
centroids = current_df.loc[:, ['column', 'row']].to_numpy()
centroids.max(axis=0)

In [ ]:
# Load pre-defined cell segmentation 
labels = load_label_image(os.path.join(seg_path, "segmentation"), fname=f'{current_sample}_labeled_cells_org.tif')
labels.shape

In [ ]:
# load overlay image 
gfap = load_nissl_image(os.path.join(seg_path, 'raw'), fname=f"{current_sample}_Gfap.tif")
neun = load_nissl_image(os.path.join(seg_path, 'raw'), fname=f"{current_sample}_NeuN.tif")

gfap_mask = load_nissl_image(os.path.join(seg_path, 'mask_test'), fname=f"{current_sample}_Gfap.tif")
neun_mask = load_nissl_image(os.path.join(seg_path, 'mask_test'), fname=f"{current_sample}_NeuN.tif")

neun.shape

In [ ]:
gfap_mask = gfap_mask > 0
neun_mask = neun_mask > 0 

In [ ]:
# get reads assignment 
reads_assignment = labels[centroids[:, 1], centroids[:, 0]]
reads_assignment

In [ ]:
%%time
        
areas = [0, ]
neun_intensity = [0, ]
gfap_intensity = [0, ]

neun_pixel = [0, ]
gfap_pixel = [0, ]

region_label = [0, ]

# Iterate through cells
print('Iterate cells...')
for i, region in enumerate(tqdm(regionprops(labels))):
      # print(region.label)
    region_label.append(region.label)
    areas.append(region.area)
    
    # neun intensity
    curr_intensity = np.sum(neun[region.coords[:, 0], region.coords[:, 1]])
    curr_pixel = np.sum(neun_mask[region.coords[:, 0], region.coords[:, 1]])
    
    neun_intensity.append(round(curr_intensity, 2))
    neun_pixel.append(round(curr_pixel, 2))
        
    # gfap intensity
    curr_intensity = np.sum(gfap[region.coords[:, 0], region.coords[:, 1]])
    curr_pixel = np.sum(gfap_mask[region.coords[:, 0], region.coords[:, 1]])
    
    gfap_intensity.append(round(curr_intensity, 2))
    gfap_pixel.append(round(curr_pixel, 2))

In [ ]:
neun_intensity = np.array(neun_intensity)
neun_pixel = np.array(neun_pixel)
gfap_intensity = np.array(gfap_intensity)
gfap_pixel = np.array(gfap_pixel)
region_label = np.array(region_label)
areas = np.array(areas)

In [ ]:
# find duplicate assignment 
from collections import Counter
dup_assignment = [item for item, count in Counter(reads_assignment).items() if count > 1]

for i in dup_assignment:
    reads_assignment[reads_assignment == i] = 0

In [ ]:
indexes = np.array([np.where(region_label == i)[0][0] for i in reads_assignment])

In [ ]:
current_df['seg_label'] = reads_assignment
current_df['area'] = areas[indexes]
current_df['Gfap'] = gfap_intensity[indexes]
current_df['NeuN'] = neun_intensity[indexes]
current_df['Gfap_pixel'] = gfap_pixel[indexes]
current_df['NeuN_pixel'] = neun_pixel[indexes]

In [ ]:
cdata.obs.loc[cdata.obs['protocol-replicate'] == f'{current_sample}map-rep2' , 'seg_label'] = current_df['seg_label']
cdata.obs.loc[cdata.obs['protocol-replicate'] == f'{current_sample}map-rep2' , 'area'] = current_df['area']
cdata.obs.loc[cdata.obs['protocol-replicate'] == f'{current_sample}map-rep2' , 'Gfap_intensity'] = current_df['Gfap']
cdata.obs.loc[cdata.obs['protocol-replicate'] == f'{current_sample}map-rep2' , 'NeuN_intensity'] = current_df['NeuN']
cdata.obs.loc[cdata.obs['protocol-replicate'] == f'{current_sample}map-rep2' , 'Gfap_pixel'] = current_df['Gfap_pixel']
cdata.obs.loc[cdata.obs['protocol-replicate'] == f'{current_sample}map-rep2' , 'NeuN_pixel'] = current_df['NeuN_pixel']

In [ ]:
from datetime import datetime
date = datetime.today().strftime('%Y-%m-%d')
cdata.write_h5ad(f"{out_path}/{date}-Brain-combined-3mad-ct-final.h5ad")